# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
import re

def extract_floats(s):
    """
    Extracts all floats from a string and returns them as a list of strings.

    Parameters:
    - s (str): The string to extract floats from.

    Returns:
    - list: A list of strings, each representing a float found in the input string.
    """
    float_pattern = r"[-+]?\d*\.\d+|\d+"
    return [str(float(num)) for num in re.findall(float_pattern, s)]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [9]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
INPUT_DIR = "/scratch/back_up/reward_competition_extention/data/rce_cohort_2"
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2_comp_omission"

In [10]:
COLS_TO_KEEP = ['session_dir', 'recording', 'metadata_dir', 'metadata_file',
'original_file', 'filename', 'session_path', 'all_subjects',
       'current_subject', 'event_timestamps', 'video_name',
       'video_timestamps', 'event_frames', 'first_item_data']

In [11]:
RAW_COLS_TO_KEEP = ['session_dir',
 'recording',
 'original_file',
 'session_path',
 'current_subject',
 'first_item_data',
 'first_timestamp',
 'all_subjects']

In [12]:
STATE_COLS_TO_KEEP = ['session_dir',
 'metadata_file',
 'event_timestamps',
 'video_name',
 'video_timestamps',
 'event_frames',]

In [13]:
same_columns = ['session_dir', 'video_name']
different_columns = ['metadata_file', 'event_frames', 'event_timestamps']

In [14]:
# TODO: Find way not to hard code this
# ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_*/*.rec")
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/rce_cohort_2/omission/*.rec")



In [15]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/rce_cohort_2/omission/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_2/omission/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_2/omission/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_2/omission/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_2/omission/20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [16]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# LOOP 1: Extracting all the Trodes

- Getting all the data from all the exported Trodes files

- Getting all the data from all the exported Trodes files and saving it to `session_to_trodes_data`
    - Creates a dictionary with the structure of:
        - `{dir_name: {file_name: metadata, file_name_2: metadata_2}, dir_name_2: {file_name_3: metadata_3, file_name_4: metadata_4}}`

In [17]:
# Saving the trodes data for each session
# Each key is a session name
# Each value is a dictionary of every recording file in that session
session_to_trodes_data = utilities.helper.create_recursive_dict()


# Saving the path of the session recording
session_to_path = {}

# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        # Getting the name of the session from the path
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        # Reading the trodes data for every recording file in the session directory
        session_to_trodes_data[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
        
        session_to_path[session_basename] = session_path
    except Exception as e: 
        print(e)


Current Session: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2


/nancy/user/riwata/projects/reward_comp_ext/src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2
Skipping file 20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1
Skipping file 20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1
Skipping fi

In [18]:
session_to_trodes_data

defaultdict(<function utilities.helper.create_recursive_dict()>,
            {'20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2': defaultdict(dict,
                         {'20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged': {'DIO': {'dio_ECU_Dout1': {'description': 'State change data for one digital channel. Display_order is 1-based',
                             'byte_order': 'little endian',
                             'original_file': '20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec',
                             'clockrate': '20000',
                             'trodes_version': '2.4.0',
                             'compile_date': 'May 24 2023',
                             'compile_time': '10:11:04',
                             'qt_version': '6.2.2',
                             'commit_tag': 'heads/Release_2.4.0-0-g1eecf3b7',
                             'controller_firmware': '3.17',
                          

- Adding the video timestamps

In [19]:
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)
            if "video_timestamps" not in session_to_trodes_data[session_basename][session_basename]:
                session_to_trodes_data[session_basename][session_basename]["video_timestamps"] = defaultdict(dict)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"][video_basename.split(".")[-3]] = timestamp_array
    
    
    except Exception as e: 
        print(e)

Current Session: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2
Current Video Name: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3.videoTimeStamps.cameraHWSync
Current Video Name: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync
Current Session: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2
Current Video Name: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.1.videoTimeStamps.cameraHWSync
Current Session: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1
Current Video Name: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1.v

In [20]:
session_to_trodes_data[session_basename][session_basename]["video_timestamps"]

defaultdict(dict,
            {'4': {'clock rate': '30000',
              'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
              'data': array([( 1642254, 0, 0), ( 1643640, 0, 0), ( 1645026, 0, 0), ...,
                     (67770406, 0, 0), (67770406, 0, 0), (67771792, 0, 0)],
                    dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')]),
              'filename': '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.4.videoTimeStamps.cameraHWSync'}})

- Creating a dataframe the dictionary with a column for:
  - Session directory
  - Recording name
  - Metadata directory
  - Metadata file
  - And a column for each metadata

In [21]:
# Creating a dataframe from the nested dictionary
trodes_metadata_df = pd.DataFrame.from_dict({(i,j,k,l): session_to_trodes_data[i][j][k][l] 
                           for i in session_to_trodes_data.keys() 
                           for j in session_to_trodes_data[i].keys()
                           for k in session_to_trodes_data[i][j].keys()
                           for l in session_to_trodes_data[i][j][k].keys()},
                           orient='index')

# Resetting the index and renaming the columns
trodes_metadata_df = trodes_metadata_df.reset_index()
trodes_metadata_df = trodes_metadata_df.rename(columns={'level_0': 'session_dir', 'level_1': 'recording', 'level_2': 'metadata_dir', 'level_3': 'metadata_file'}, errors="ignore")

# Adding the session path to the dataframe
trodes_metadata_df["session_path"] = trodes_metadata_df["session_dir"].map(session_to_path)

In [22]:
trodes_metadata_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                           recording metadata_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
1  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
2  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
3  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
4  20230617_115521_standard_comp_to_omission_D1_s...          DIO   

   metadata_file                                        description  \
0  dio_ECU_Dout1  State change data for one digital channel. Dis...   
1  dio_ECU_Dout2  State change data for one digital channel. Dis...   
2   dio_ECU_Din1  State change data for one digital channel. Dis...   
3  dio_ECU_Dout4  State change data for one digital channel. Dis...   
4   dio_ECU_Din2  State change data for one digital channel. Dis...   

      byte_order                                      original_file clockrate  \
0  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
1  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
2  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
3  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
4  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   

  trodes_version compile_date  ... first_timestamp direction         id  \
0          2.4.0  May 24 2023  ...         3478533    output  ECU_Dout1   
1          2.4.0  May 24 2023  ...         3478533    output  ECU_Dout2   
2          2.4.0  May 24 2023  ...         3478533     input   ECU_Din1   
3          2.4.0  May 24 2023  ...         3478533    output  ECU_Dout4   
4          2.4.0  May 24 2023  ...         3478533     input   ECU_Din2   

  display_order                      fields  \
0             2  <time uint32><state uint8>   
1             3  <time uint32><state uint8>   
2             7  <time uint32><state uint8>   
3             5  <time uint32><state uint8>   
4             6  <time uint32><state uint8>   

                                                data  \
0                                     [[3478533, 0]]   
1                                     [[3478533, 0]]   
2  [[3478533, 0], [4509412, 1], [4709411, 0], [69...   
3                                     [[3478533, 0]]   
4  [[3478533, 0], [3581011, 1], [3583414, 0], [35...   

                                            filename decimation clock rate  \
0  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
1  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
2  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
3  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
4  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   

                                        session_path  
0  /scratch/back_up/reward_competition_extention/...  
1  /scratch/back_up/reward_competition_extention/...  
2  /scratch/back_up/reward_competition_extention/...  
3  /scratch/back_up/reward_competition_extention/...  
4  /scratch/back_up/reward_competition_extention/...  

[5 rows x 35 columns]

In [23]:
trodes_metadata_df.tail()

session_dir  \
100  20230620_114347_standard_comp_to_omission_D4_s...   
101  20230620_114347_standard_comp_to_omission_D4_s...   
102  20230618_100636_standard_comp_to_omission_D2_s...   
103  20230618_100636_standard_comp_to_omission_D2_s...   
104  20230619_115321_standard_comp_to_omission_D3_s...   

                                             recording      metadata_dir  \
100  20230620_114347_standard_comp_to_omission_D4_s...  video_timestamps   
101  20230620_114347_standard_comp_to_omission_D4_s...  video_timestamps   
102  20230618_100636_standard_comp_to_omission_D2_s...  video_timestamps   
103  20230618_100636_standard_comp_to_omission_D2_s...  video_timestamps   
104  20230619_115321_standard_comp_to_omission_D3_s...  video_timestamps   

    metadata_file description byte_order original_file clockrate  \
100             2         NaN        NaN           NaN       NaN   
101             1         NaN        NaN           NaN       NaN   
102             1         NaN        NaN           NaN       NaN   
103             2         NaN        NaN           NaN       NaN   
104             4         NaN        NaN           NaN       NaN   

    trodes_version compile_date  ... first_timestamp direction   id  \
100            NaN          NaN  ...             NaN       NaN  NaN   
101            NaN          NaN  ...             NaN       NaN  NaN   
102            NaN          NaN  ...             NaN       NaN  NaN   
103            NaN          NaN  ...             NaN       NaN  NaN   
104            NaN          NaN  ...             NaN       NaN  NaN   

    display_order                                             fields  \
100           NaN  <PosTimestamp uint32><HWframeCount uint32><HWT...   
101           NaN  <PosTimestamp uint32><HWframeCount uint32><HWT...   
102           NaN  <PosTimestamp uint32><HWframeCount uint32><HWT...   
103           NaN  <PosTimestamp uint32><HWframeCount uint32><HWT...   
104           NaN  <PosTimestamp uint32><HWframeCount uint32><HWT...   

                                                  data  \
100  [[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...   
101  [[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...   
102  [[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...   
103  [[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...   
104  [[1642254, 0, 0], [1643640, 0, 0], [1645026, 0...   

                                              filename decimation clock rate  \
100  20230620_114347_standard_comp_to_omission_D4_s...        NaN      30000   
101  20230620_114347_standard_comp_to_omission_D4_s...        NaN      30000   
102  20230618_100636_standard_comp_to_omission_D2_s...        NaN      30000   
103  20230618_100636_standard_comp_to_omission_D2_s...        NaN      30000   
104  20230619_115321_standard_comp_to_omission_D3_s...        NaN      30000   

                                          session_path  
100  /scratch/back_up/reward_competition_extention/...  
101  /scratch/back_up/reward_competition_extention/...  
102  /scratch/back_up/reward_competition_extention/...  
103  /scratch/back_up/reward_competition_extention/...  
104  /scratch/back_up/reward_competition_extention/...  

[5 rows x 35 columns]

- Getting the first item from each tuple in the arrays in the `data` column
  - This first item is usually just the timestamp

In [24]:
trodes_metadata_df["data"].iloc[0]

array([(3478533, 0)], dtype=[('time', '<u4'), ('state', 'u1')])

In [25]:
# Getting the dtype name of each column in the numpy array
trodes_metadata_df["first_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[0])
# Getting the first item of each column in the numpy array
trodes_metadata_df["first_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[0]])


In [26]:
# Same as above but for the last column
trodes_metadata_df["last_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[-1])
trodes_metadata_df["last_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[-1]])

In [27]:
trodes_metadata_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                           recording metadata_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
1  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
2  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
3  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
4  20230617_115521_standard_comp_to_omission_D1_s...          DIO   

   metadata_file                                        description  \
0  dio_ECU_Dout1  State change data for one digital channel. Dis...   
1  dio_ECU_Dout2  State change data for one digital channel. Dis...   
2   dio_ECU_Din1  State change data for one digital channel. Dis...   
3  dio_ECU_Dout4  State change data for one digital channel. Dis...   
4   dio_ECU_Din2  State change data for one digital channel. Dis...   

      byte_order                                      original_file clockrate  \
0  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
1  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
2  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
3  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
4  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   

  trodes_version compile_date  ...                      fields  \
0          2.4.0  May 24 2023  ...  <time uint32><state uint8>   
1          2.4.0  May 24 2023  ...  <time uint32><state uint8>   
2          2.4.0  May 24 2023  ...  <time uint32><state uint8>   
3          2.4.0  May 24 2023  ...  <time uint32><state uint8>   
4          2.4.0  May 24 2023  ...  <time uint32><state uint8>   

                                                data  \
0                                     [[3478533, 0]]   
1                                     [[3478533, 0]]   
2  [[3478533, 0], [4509412, 1], [4709411, 0], [69...   
3                                     [[3478533, 0]]   
4  [[3478533, 0], [3581011, 1], [3583414, 0], [35...   

                                            filename decimation clock rate  \
0  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
1  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
2  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
3  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
4  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   

                                        session_path first_dtype_name  \
0  /scratch/back_up/reward_competition_extention/...             time   
1  /scratch/back_up/reward_competition_extention/...             time   
2  /scratch/back_up/reward_competition_extention/...             time   
3  /scratch/back_up/reward_competition_extention/...             time   
4  /scratch/back_up/reward_competition_extention/...             time   

                                     first_item_data last_dtype_name  \
0                                          [3478533]           state   
1                                          [3478533]           state   
2  [3478533, 4509412, 4709411, 6909411, 7109411, ...           state   
3                                          [3478533]           state   
4  [3478533, 3581011, 3583414, 3589014, 3626412, ...           state   

                                      last_item_data  
0                                                [0]  
1                                                [0]  
2  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...  
3                                                [0]  
4  [0, 1, 0, 1, 0, 1, 0, 

In [28]:
trodes_metadata_df.tail()

session_dir  \
100  20230620_114347_standard_comp_to_omission_D4_s...   
101  20230620_114347_standard_comp_to_omission_D4_s...   
102  20230618_100636_standard_comp_to_omission_D2_s...   
103  20230618_100636_standard_comp_to_omission_D2_s...   
104  20230619_115321_standard_comp_to_omission_D3_s...   

                                             recording      metadata_dir  \
100  20230620_114347_standard_comp_to_omission_D4_s...  video_timestamps   
101  20230620_114347_standard_comp_to_omission_D4_s...  video_timestamps   
102  20230618_100636_standard_comp_to_omission_D2_s...  video_timestamps   
103  20230618_100636_standard_comp_to_omission_D2_s...  video_timestamps   
104  20230619_115321_standard_comp_to_omission_D3_s...  video_timestamps   

    metadata_file description byte_order original_file clockrate  \
100             2         NaN        NaN           NaN       NaN   
101             1         NaN        NaN           NaN       NaN   
102             1         NaN        NaN           NaN       NaN   
103             2         NaN        NaN           NaN       NaN   
104             4         NaN        NaN           NaN       NaN   

    trodes_version compile_date  ...  \
100            NaN          NaN  ...   
101            NaN          NaN  ...   
102            NaN          NaN  ...   
103            NaN          NaN  ...   
104            NaN          NaN  ...   

                                                fields  \
100  <PosTimestamp uint32><HWframeCount uint32><HWT...   
101  <PosTimestamp uint32><HWframeCount uint32><HWT...   
102  <PosTimestamp uint32><HWframeCount uint32><HWT...   
103  <PosTimestamp uint32><HWframeCount uint32><HWT...   
104  <PosTimestamp uint32><HWframeCount uint32><HWT...   

                                                  data  \
100  [[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...   
101  [[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...   
102  [[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...   
103  [[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...   
104  [[1642254, 0, 0], [1643640, 0, 0], [1645026, 0...   

                                              filename decimation clock rate  \
100  20230620_114347_standard_comp_to_omission_D4_s...        NaN      30000   
101  20230620_114347_standard_comp_to_omission_D4_s...        NaN      30000   
102  20230618_100636_standard_comp_to_omission_D2_s...        NaN      30000   
103  20230618_100636_standard_comp_to_omission_D2_s...        NaN      30000   
104  20230619_115321_standard_comp_to_omission_D3_s...        NaN      30000   

                                          session_path first_dtype_name  \
100  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
101  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
102  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
103  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
104  /scratch/back_up/reward_competition_extention/...     PosTimestamp   

                                       first_item_data last_dtype_name  \
100  [2504265, 2504265, 2505651, 2507037, 2508422, ...     HWTimestamp   
101  [2504265, 2504265, 2505651, 2507037, 2508422, ...     HWTimestamp   
102  [835678, 837064, 838450, 838450, 839835, 84122...     HWTimestamp   
103  [835678, 837064, 838450, 838921, 839835, 84122...     HWTimestamp   
104  [1642254, 1643640, 1645026, 1645026, 1646412, ...     HWTimestamp   

                                        last_item_data  
100  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
101  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
102  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
103  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
104  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  

[5 rows x 39 columns]

In [29]:
trodes_metadata_df["recording"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_s

## Getting the subject information from the metadata

In [30]:
def split_by_multiple_delimiters(s, delimiters):
    """
    Splits a string by multiple delimiters.

    Parameters:
    - s (str): The string to split.
    - delimiters (list): A list of delimiters to split the string by.

    Returns:
    - list: A list of substrings.
    """
    return re.split('|'.join(map(re.escape, delimiters)), s)


In [31]:
trodes_metadata_df["all_subjects"] = trodes_metadata_df["session_dir"].apply(lambda x: x.split("subj")[-1].strip("_").replace("-", "."))#.split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["all_subjects"] = trodes_metadata_df["all_subjects"].apply(lambda x: sorted(extract_floats(x)))

In [32]:
trodes_metadata_df["session_dir"].iloc[0]

'20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2'

In [33]:
trodes_metadata_df["all_subjects"].apply(lambda x: tuple(x)).unique()

array([('1.1', '1.2'), ('1.2', '1.4'), ('1.1', '1.4')], dtype=object)

In [34]:
trodes_metadata_df["current_subject"] = trodes_metadata_df["recording"].apply(lambda x: x.split("subj")[-1].strip("_").replace("-", ".").replace("_", "."))#.split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["current_subject"] = trodes_metadata_df["current_subject"].apply(lambda x: str(extract_floats(x)[0]).strip())


In [35]:
trodes_metadata_df["current_subject"].unique()

array(['1.1', '1.2', '1.4'], dtype=object)

## Dropping all the rows with unneeded metadata

In [36]:
trodes_metadata_df["metadata_dir"].unique()

array(['DIO', 'raw', 'time', 'analog', 'video_timestamps'], dtype=object)

In [37]:
METADATA_TO_KEEP = ['raw', 'DIO', 'video_timestamps']

In [38]:
trodes_metadata_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(METADATA_TO_KEEP)]

In [39]:
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("out")]
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("coordinates")]


In [40]:
trodes_metadata_df = trodes_metadata_df.reset_index(drop=True)

# Getting the first time stamp of each recording

In [41]:
trodes_raw_df = trodes_metadata_df[(trodes_metadata_df["metadata_dir"] == "raw") & (trodes_metadata_df["metadata_file"] == "timestamps")].copy()


In [42]:
trodes_raw_df.head()

session_dir  \
4   20230617_115521_standard_comp_to_omission_D1_s...   
9   20230617_115521_standard_comp_to_omission_D1_s...   
10  20230621_111240_standard_comp_to_omission_D5_s...   
19  20230620_114347_standard_comp_to_omission_D4_s...   
24  20230620_114347_standard_comp_to_omission_D4_s...   

                                            recording metadata_dir  \
4   20230617_115521_standard_comp_to_omission_D1_s...          raw   
9   20230617_115521_standard_comp_to_omission_D1_s...          raw   
10  20230621_111240_standard_comp_to_omission_D5_s...          raw   
19  20230620_114347_standard_comp_to_omission_D4_s...          raw   
24  20230620_114347_standard_comp_to_omission_D4_s...          raw   

   metadata_file     description     byte_order  \
4     timestamps  Raw timestamps  little endian   
9     timestamps  Raw timestamps  little endian   
10    timestamps  Raw timestamps  little endian   
19    timestamps  Raw timestamps  little endian   
24    timestamps  Raw timestamps  little endian   

                                        original_file clockrate  \
4   20230617_115521_standard_comp_to_omission_D1_s...     20000   
9   20230617_115521_standard_comp_to_omission_D1_s...     20000   
10  20230621_111240_standard_comp_to_omission_D5_s...     20000   
19  20230620_114347_standard_comp_to_omission_D4_s...     20000   
24  20230620_114347_standard_comp_to_omission_D4_s...     20000   

   trodes_version compile_date  ...  \
4           2.4.0  May 24 2023  ...   
9           2.4.0  May 24 2023  ...   
10          2.4.0  May 24 2023  ...   
19          2.4.0  May 24 2023  ...   
24          2.4.0  May 24 2023  ...   

                                             filename decimation clock rate  \
4   20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
9   20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
10  20230621_111240_standard_comp_to_omission_D5_s...        NaN        NaN   
19  20230620_114347_standard_comp_to_omission_D4_s...        NaN        NaN   
24  20230620_114347_standard_comp_to_omission_D4_s...        NaN        NaN   

                                         session_path first_dtype_name  \
4   /scratch/back_up/reward_competition_extention/...             time   
9   /scratch/back_up/reward_competition_extention/...             time   
10  /scratch/back_up/reward_competition_extention/...             time   
19  /scratch/back_up/reward_competition_extention/...             time   
24  /scratch/back_up/reward_competition_extention/...             time   

                                      first_item_data last_dtype_name  \
4   [3478533, 3478534, 3478535, 3478536, 3478537, ...            time   
9   [3478533, 3478534, 3478535, 3478536, 3478537, ...            time   
10  [3193044, 3193045, 3193046, 3193047, 3193048, ...            time   
19  [2502881, 2502882, 2502883, 2502884, 2502885, ...            time   
24  [2502881, 2502882, 2502883, 2502884, 2502885, ...            time   

                                       last_item_data all_subjects  \
4   [3478533, 3478534, 3478535, 3478536, 3478537, ...   [1.1, 1.2]   
9   [3478533, 3478534, 3478535, 3478536, 3478537, ...   [1.1, 1.2]   
10  [3193044, 3193045, 3193046, 3193047, 3193048, ...   [1.2, 1.4]   
19  [2502881, 2502882, 2502883, 2502884, 2502885, ...   [1.1, 1.2]   
24  [2502881, 2502882, 2502883, 2502884, 2502885, ...   [1.1, 1.2]   

   current_subject  
4              1.1  
9              1.2  
10             1.4  
19             1.1  
24             1.2  

[5 rows x 41 columns]

In [43]:
trodes_raw_df["first_timestamp"] = trodes_raw_df["first_item_data"].apply(lambda x: x[0])

In [44]:
trodes_raw_df["recording"].iloc[0]

'20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged'

In [45]:
recording_to_first_timestamp = trodes_raw_df.set_index('session_dir')['first_timestamp'].to_dict()

In [46]:
recording_to_first_timestamp

{'20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2': 3478533,
 '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2': 3193044,
 '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1': 2502881,
 '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1': 835680,
 '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4': 1642256}

In [47]:
trodes_metadata_df["first_timestamp"] = trodes_metadata_df["session_dir"].map(recording_to_first_timestamp)

In [48]:
trodes_metadata_df["first_timestamp"]

0     3478533
1     3478533
2     3478533
3     3478533
4     3478533
5     3478533
6     3478533
7     3478533
8     3478533
9     3478533
10    3193044
11    3193044
12    3193044
13    3193044
14    3193044
15    2502881
16    2502881
17    2502881
18    2502881
19    2502881
20    2502881
21    2502881
22    2502881
23    2502881
24    2502881
25     835680
26     835680
27     835680
28     835680
29     835680
30     835680
31     835680
32     835680
33     835680
34     835680
35    1642256
36    1642256
37    1642256
38    1642256
39    1642256
40    3478533
41    3478533
42    3478533
43    3193044
44    3193044
45    2502881
46    2502881
47     835680
48     835680
49    1642256
Name: first_timestamp, dtype: int64

# Getting the event timestamps

In [49]:
trodes_metadata_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                           recording metadata_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
1  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
2  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
3  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
4  20230617_115521_standard_comp_to_omission_D1_s...          raw   

  metadata_file                                        description  \
0  dio_ECU_Din1  State change data for one digital channel. Dis...   
1  dio_ECU_Din2  State change data for one digital channel. Dis...   
2  dio_ECU_Din3  State change data for one digital channel. Dis...   
3  dio_ECU_Din4  State change data for one digital channel. Dis...   
4    timestamps                                     Raw timestamps   

      byte_order                                      original_file clockrate  \
0  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
1  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
2  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
3  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
4  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   

  trodes_version compile_date  ...  \
0          2.4.0  May 24 2023  ...   
1          2.4.0  May 24 2023  ...   
2          2.4.0  May 24 2023  ...   
3          2.4.0  May 24 2023  ...   
4          2.4.0  May 24 2023  ...   

                                            filename decimation clock rate  \
0  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
1  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
2  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
3  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
4  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   

                                        session_path first_dtype_name  \
0  /scratch/back_up/reward_competition_extention/...             time   
1  /scratch/back_up/reward_competition_extention/...             time   
2  /scratch/back_up/reward_competition_extention/...             time   
3  /scratch/back_up/reward_competition_extention/...             time   
4  /scratch/back_up/reward_competition_extention/...             time   

                                     first_item_data last_dtype_name  \
0  [3478533, 4509412, 4709411, 6909411, 7109411, ...           state   
1  [3478533, 3581011, 3583414, 3589014, 3626412, ...           state   
2  [3478533, 28509009, 34858606, 43086003, 430998...           state   
3                                          [3478533]           state   
4  [3478533, 3478534, 3478535, 3478536, 3478537, ...            time   

                                      last_item_data all_subjects  \
0  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
1  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
2  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
3                                                [0]   [1.1, 1.2]   
4  [3478533, 3478534, 3478535, 3478536, 3478537, ...   [1.1, 1.2]   

  current_subject  
0             1.1  
1             1.1  
2             1.1  
3             1.1  
4             1.1  

[5 rows x 41 columns]

In [50]:
trodes_metadata_df.tail()

session_dir  \
45  20230620_114347_standard_comp_to_omission_D4_s...   
46  20230620_114347_standard_comp_to_omission_D4_s...   
47  20230618_100636_standard_comp_to_omission_D2_s...   
48  20230618_100636_standard_comp_to_omission_D2_s...   
49  20230619_115321_standard_comp_to_omission_D3_s...   

                                            recording      metadata_dir  \
45  20230620_114347_standard_comp_to_omission_D4_s...  video_timestamps   
46  20230620_114347_standard_comp_to_omission_D4_s...  video_timestamps   
47  20230618_100636_standard_comp_to_omission_D2_s...  video_timestamps   
48  20230618_100636_standard_comp_to_omission_D2_s...  video_timestamps   
49  20230619_115321_standard_comp_to_omission_D3_s...  video_timestamps   

   metadata_file description byte_order original_file clockrate  \
45             2         NaN        NaN           NaN       NaN   
46             1         NaN        NaN           NaN       NaN   
47             1         NaN        NaN           NaN       NaN   
48             2         NaN        NaN           NaN       NaN   
49             4         NaN        NaN           NaN       NaN   

   trodes_version compile_date  ...  \
45            NaN          NaN  ...   
46            NaN          NaN  ...   
47            NaN          NaN  ...   
48            NaN          NaN  ...   
49            NaN          NaN  ...   

                                             filename decimation clock rate  \
45  20230620_114347_standard_comp_to_omission_D4_s...        NaN      30000   
46  20230620_114347_standard_comp_to_omission_D4_s...        NaN      30000   
47  20230618_100636_standard_comp_to_omission_D2_s...        NaN      30000   
48  20230618_100636_standard_comp_to_omission_D2_s...        NaN      30000   
49  20230619_115321_standard_comp_to_omission_D3_s...        NaN      30000   

                                         session_path first_dtype_name  \
45  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
46  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
47  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
48  /scratch/back_up/reward_competition_extention/...     PosTimestamp   
49  /scratch/back_up/reward_competition_extention/...     PosTimestamp   

                                      first_item_data last_dtype_name  \
45  [2504265, 2504265, 2505651, 2507037, 2508422, ...     HWTimestamp   
46  [2504265, 2504265, 2505651, 2507037, 2508422, ...     HWTimestamp   
47  [835678, 837064, 838450, 838450, 839835, 84122...     HWTimestamp   
48  [835678, 837064, 838450, 838921, 839835, 84122...     HWTimestamp   
49  [1642254, 1643640, 1645026, 1645026, 1646412, ...     HWTimestamp   

                                       last_item_data all_subjects  \
45  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   [1.1, 1.2]   
46  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   [1.1, 1.2]   
47  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   [1.1, 1.4]   
48  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   [1.1, 1.4]   
49  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   [1.2, 1.4]   

   current_subject  
45             1.2  
46             1.2  
47             1.4  
48             1.4  
49             1.2  

[5 rows x 41 columns]

In [51]:
# trodes_state_df = trodes_metadata_df[trodes_metadata_df["last_dtype_name"] == "state"].copy()

# Filtering for digital IO channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(["DIO"])].copy()
# Filtering for tone and port entry related channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["id"].isin(["ECU_Din1", "ECU_Din2", "ECU_Din3"])].copy()


In [52]:
trodes_state_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
6  20230617_115521_standard_comp_to_omission_D1_s...   
7  20230617_115521_standard_comp_to_omission_D1_s...   

                                           recording metadata_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
1  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
2  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
6  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
7  20230617_115521_standard_comp_to_omission_D1_s...          DIO   

  metadata_file                                        description  \
0  dio_ECU_Din1  State change data for one digital channel. Dis...   
1  dio_ECU_Din2  State change data for one digital channel. Dis...   
2  dio_ECU_Din3  State change data for one digital channel. Dis...   
6  dio_ECU_Din3  State change data for one digital channel. Dis...   
7  dio_ECU_Din2  State change data for one digital channel. Dis...   

      byte_order                                      original_file clockrate  \
0  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
1  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
2  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
6  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
7  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   

  trodes_version compile_date  ...  \
0          2.4.0  May 24 2023  ...   
1          2.4.0  May 24 2023  ...   
2          2.4.0  May 24 2023  ...   
6          2.4.0  May 24 2023  ...   
7          2.4.0  May 24 2023  ...   

                                            filename decimation clock rate  \
0  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
1  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
2  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
6  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   
7  20230617_115521_standard_comp_to_omission_D1_s...        NaN        NaN   

                                        session_path first_dtype_name  \
0  /scratch/back_up/reward_competition_extention/...             time   
1  /scratch/back_up/reward_competition_extention/...             time   
2  /scratch/back_up/reward_competition_extention/...             time   
6  /scratch/back_up/reward_competition_extention/...             time   
7  /scratch/back_up/reward_competition_extention/...             time   

                                     first_item_data last_dtype_name  \
0  [3478533, 4509412, 4709411, 6909411, 7109411, ...           state   
1  [3478533, 3581011, 3583414, 3589014, 3626412, ...           state   
2  [3478533, 28509009, 34858606, 43086003, 430998...           state   
6  [3478533, 28509009, 34858606, 43086003, 430998...           state   
7  [3478533, 3581011, 3583414, 3589014, 3626412, ...           state   

                                      last_item_data all_subjects  \
0  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
1  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
2  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
6  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
7  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   

  current_subject  
0             1.1  
1             1.1  
2             1.1  
6             1.2  
7             1.2  

[5 rows x 41 columns]

In [53]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: np.column_stack([np.where(x["last_item_data"] == 1)[0], np.where(x["last_item_data"] == 1)[0]+1]), axis=1)

In [54]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: x["event_indexes"][x["event_indexes"][:, 1] <= x["first_item_data"].shape[0] - 1], axis=1)

In [55]:
trodes_state_df["event_timestamps"] = trodes_state_df.apply(lambda x: x["first_item_data"][x["event_indexes"]], axis=1)

## Updating the video timestamps

## Syncing up the video frame data

In [56]:
# Getting the rows that are the metadata for the video timestamps
trodes_video_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"] == "video_timestamps"].copy().reset_index(drop=True)



In [57]:
# Filtering for the first video only
# This only applies to this pilot data where we are only looking the at competition data
# trodes_video_df = trodes_video_df[trodes_video_df["metadata_file"] == "1"].copy()

In [58]:
trodes_video_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230621_111240_standard_comp_to_omission_D5_s...   
4  20230621_111240_standard_comp_to_omission_D5_s...   

                                           recording      metadata_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...  video_timestamps   
1  20230617_115521_standard_comp_to_omission_D1_s...  video_timestamps   
2  20230617_115521_standard_comp_to_omission_D1_s...  video_timestamps   
3  20230621_111240_standard_comp_to_omission_D5_s...  video_timestamps   
4  20230621_111240_standard_comp_to_omission_D5_s...  video_timestamps   

  metadata_file description byte_order original_file clockrate trodes_version  \
0             3         NaN        NaN           NaN       NaN            NaN   
1             2         NaN        NaN           NaN       NaN            NaN   
2             1         NaN        NaN           NaN       NaN            NaN   
3             2         NaN        NaN           NaN       NaN            NaN   
4             1         NaN        NaN           NaN       NaN            NaN   

  compile_date  ...                                           filename  \
0          NaN  ...  20230617_115521_standard_comp_to_omission_D1_s...   
1          NaN  ...  20230617_115521_standard_comp_to_omission_D1_s...   
2          NaN  ...  20230617_115521_standard_comp_to_omission_D1_s...   
3          NaN  ...  20230621_111240_standard_comp_to_omission_D5_s...   
4          NaN  ...  20230621_111240_standard_comp_to_omission_D5_s...   

  decimation clock rate                                       session_path  \
0        NaN      30000  /scratch/back_up/reward_competition_extention/...   
1        NaN      30000  /scratch/back_up/reward_competition_extention/...   
2        NaN      30000  /scratch/back_up/reward_competition_extention/...   
3        NaN      30000  /scratch/back_up/reward_competition_extention/...   
4        NaN      30000  /scratch/back_up/reward_competition_extention/...   

  first_dtype_name                                    first_item_data  \
0     PosTimestamp  [44148259, 44149645, 44149645, 44151031, 44152...   
1     PosTimestamp  [3478531, 3479917, 3479917, 3481303, 3482689, ...   
2     PosTimestamp  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
3     PosTimestamp  [3193042, 3194428, 3195138, 3195814, 3197200, ...   
4     PosTimestamp  [3193042, 3194428, 3194428, 3195814, 3197200, ...   

  last_dtype_name                                     last_item_data  \
0     HWTimestamp  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1     HWTimestamp  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2     HWTimestamp  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3     HWTimestamp  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4     HWTimestamp  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

  all_subjects current_subject  
0   [1.1, 1.2]             1.1  
1   [1.1, 1.2]             1.1  
2   [1.1, 1.2]             1.1  
3   [1.2, 1.4]             1.4  
4   [1.2, 1.4]             1.4  

[5 rows x 41 columns]

In [59]:
# Making the video timestamps into an evenly distributed array
trodes_video_df["video_timestamps"] = trodes_video_df["first_item_data"]

In [60]:
# Removing the columns that are no longer needed
trodes_video_df = trodes_video_df[["filename", "video_timestamps", "session_dir"]].copy()

In [61]:
# Renaming the filename so that we can merge with other dataframes with the same column name
trodes_video_df = trodes_video_df.rename(columns={"filename": "video_name"})

In [62]:
trodes_video_df.head()

video_name  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230621_111240_standard_comp_to_omission_D5_s...   
4  20230621_111240_standard_comp_to_omission_D5_s...   

                                    video_timestamps  \
0  [44148259, 44149645, 44149645, 44151031, 44152...   
1  [3478531, 3479917, 3479917, 3481303, 3482689, ...   
2  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
3  [3193042, 3194428, 3195138, 3195814, 3197200, ...   
4  [3193042, 3194428, 3194428, 3195814, 3197200, ...   

                                         session_dir  
0  20230617_115521_standard_comp_to_omission_D1_s...  
1  20230617_115521_standard_comp_to_omission_D1_s...  
2  20230617_115521_standard_comp_to_omission_D1_s...  
3  20230621_111240_standard_comp_to_omission_D5_s...  
4  20230621_111240_standard_comp_to_omission_D5_s...

- Adding each video as a row to each state row

In [63]:
trodes_state_df = pd.merge(trodes_state_df, trodes_video_df, on=["session_dir"], how="inner")

In [64]:
trodes_state_df.columns

Index(['session_dir', 'recording', 'metadata_dir', 'metadata_file',
       'description', 'byte_order', 'original_file', 'clockrate',
       'trodes_version', 'compile_date', 'compile_time', 'qt_version',
       'commit_tag', 'controller_firmware', 'headstage_firmware',
       'controller_serialnum', 'headstage_serialnum', 'autosettle', 'smartref',
       'gyro', 'accelerometer', 'magnetometer', 'time_offset',
       'system_time_at_creation', 'timestamp_at_creation', 'first_timestamp',
       'direction', 'id', 'display_order', 'fields', 'data', 'filename',
       'decimation', 'clock rate', 'session_path', 'first_dtype_name',
       'first_item_data', 'last_dtype_name', 'last_item_data', 'all_subjects',
       'current_subject', 'event_indexes', 'event_timestamps', 'video_name',
       'video_timestamps'],
      dtype='object')

## Finding the closest frame to each event

In [65]:
trodes_state_df["event_timestamps"].iloc[1]

array([[ 4509412,  4709411],
       [ 6909411,  7109411],
       [ 9209413,  9409413],
       [11009410, 11209410],
       [12109413, 12309412],
       [13109412, 13309412],
       [15009409, 15209412],
       [16509409, 16709411],
       [18609411, 18809411],
       [19609411, 19809408],
       [21209408, 21409408],
       [22509408, 22709408],
       [23809410, 24009408],
       [24809407, 25009408],
       [26209407, 26409407],
       [28109407, 28309409],
       [28509009, 34858606],
       [36059007, 36259008],
       [37659005, 37859005],
       [39059007, 39259004],
       [41259006, 41459006],
       [42459006, 42659003],
       [48459004, 48659004],
       [49659004, 49859001],
       [50759001, 50959001],
       [52059003, 52259003],
       [53159008, 53359003],
       [54159003, 54359000],
       [55159003, 55359000],
       [57258999, 57459004],
       [59658999, 59859001],
       [62058998, 62258998],
       [64258998, 64458997],
       [65959002, 66158999],
       [683589

In [66]:
trodes_state_df["event_frames"] = trodes_state_df.apply(lambda x: utilities.helper.find_nearest_indices(x["event_timestamps"], x["video_timestamps"]), axis=1)

In [67]:
trodes_state_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                           recording metadata_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
1  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
2  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
3  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
4  20230617_115521_standard_comp_to_omission_D1_s...          DIO   

  metadata_file                                        description  \
0  dio_ECU_Din1  State change data for one digital channel. Dis...   
1  dio_ECU_Din1  State change data for one digital channel. Dis...   
2  dio_ECU_Din1  State change data for one digital channel. Dis...   
3  dio_ECU_Din2  State change data for one digital channel. Dis...   
4  dio_ECU_Din2  State change data for one digital channel. Dis...   

      byte_order                                      original_file clockrate  \
0  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
1  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
2  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
3  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   
4  little endian  20230617_115521_standard_comp_to_omission_D1_s...     20000   

  trodes_version compile_date  ...  \
0          2.4.0  May 24 2023  ...   
1          2.4.0  May 24 2023  ...   
2          2.4.0  May 24 2023  ...   
3          2.4.0  May 24 2023  ...   
4          2.4.0  May 24 2023  ...   

                                     first_item_data last_dtype_name  \
0  [3478533, 4509412, 4709411, 6909411, 7109411, ...           state   
1  [3478533, 4509412, 4709411, 6909411, 7109411, ...           state   
2  [3478533, 4509412, 4709411, 6909411, 7109411, ...           state   
3  [3478533, 3581011, 3583414, 3589014, 3626412, ...           state   
4  [3478533, 3581011, 3583414, 3589014, 3626412, ...           state   

                                      last_item_data all_subjects  \
0  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
1  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
2  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
3  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   
4  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...   [1.1, 1.2]   

  current_subject                                      event_indexes  \
0             1.1  [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...   
1             1.1  [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...   
2             1.1  [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...   
3             1.1  [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...   
4             1.1  [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...   

                                    event_timestamps  \
0  [[4509412, 4709411], [6909411, 7109411], [9209...   
1  [[4509412, 4709411], [6909411, 7109411], [9209...   
2  [[4509412, 4709411], [6909411, 7109411], [9209...   
3  [[3581011, 3583414], [3589014, 3626412], [3671...   
4  [[3581011, 3583414], [3589014, 3626412], [3671...   

                                          video_name  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                    video_timestamps  \
0  [44148259, 44149645, 44149645, 44151031, 44152...   
1  [3478531, 3479917, 3479917, 3481303, 3482689, ...   
2  [3478531, 3479917, 3480033

## Combine raw and state dataframes

In [68]:
trodes_state_df

session_dir  \
0   20230617_115521_standard_comp_to_omission_D1_s...   
1   20230617_115521_standard_comp_to_omission_D1_s...   
2   20230617_115521_standard_comp_to_omission_D1_s...   
3   20230617_115521_standard_comp_to_omission_D1_s...   
4   20230617_115521_standard_comp_to_omission_D1_s...   
5   20230617_115521_standard_comp_to_omission_D1_s...   
6   20230617_115521_standard_comp_to_omission_D1_s...   
7   20230617_115521_standard_comp_to_omission_D1_s...   
8   20230617_115521_standard_comp_to_omission_D1_s...   
9   20230617_115521_standard_comp_to_omission_D1_s...   
10  20230617_115521_standard_comp_to_omission_D1_s...   
11  20230617_115521_standard_comp_to_omission_D1_s...   
12  20230617_115521_standard_comp_to_omission_D1_s...   
13  20230617_115521_standard_comp_to_omission_D1_s...   
14  20230617_115521_standard_comp_to_omission_D1_s...   
15  20230617_115521_standard_comp_to_omission_D1_s...   
16  20230617_115521_standard_comp_to_omission_D1_s...   
17  20230617_115521_standard_comp_to_omission_D1_s...   
18  20230621_111240_standard_comp_to_omission_D5_s...   
19  20230621_111240_standard_comp_to_omission_D5_s...   
20  20230621_111240_standard_comp_to_omission_D5_s...   
21  20230621_111240_standard_comp_to_omission_D5_s...   
22  20230621_111240_standard_comp_to_omission_D5_s...   
23  20230621_111240_standard_comp_to_omission_D5_s...   
24  20230620_114347_standard_comp_to_omission_D4_s...   
25  20230620_114347_standard_comp_to_omission_D4_s...   
26  20230620_114347_standard_comp_to_omission_D4_s...   
27  20230620_114347_standard_comp_to_omission_D4_s...   
28  20230620_114347_standard_comp_to_omission_D4_s...   
29  20230620_114347_standard_comp_to_omission_D4_s...   
30  20230620_114347_standard_comp_to_omission_D4_s...   
31  20230620_114347_standard_comp_to_omission_D4_s...   
32  20230620_114347_standard_comp_to_omission_D4_s...   
33  20230620_114347_standard_comp_to_omission_D4_s...   
34  20230620_114347_standard_comp_to_omission_D4_s...   
35  20230620_114347_standard_comp_to_omission_D4_s...   
36  20230618_100636_standard_comp_to_omission_D2_s...   
37  20230618_100636_standard_comp_to_omission_D2_s...   
38  20230618_100636_standard_comp_to_omission_D2_s...   
39  20230618_100636_standard_comp_to_omission_D2_s...   
40  20230618_100636_standard_comp_to_omission_D2_s...   
41  20230618_100636_standard_comp_to_omission_D2_s...   
42  20230618_100636_standard_comp_to_omission_D2_s...   
43  20230618_100636_standard_comp_to_omission_D2_s...   
44  20230618_100636_standard_comp_to_omission_D2_s...   
45  20230618_100636_standard_comp_to_omission_D2_s...   
46  20230618_100636_standard_comp_to_omission_D2_s...   
47  20230618_100636_standard_comp_to_omission_D2_s...   
48  20230619_115321_standard_comp_to_omission_D3_s...   
49  20230619_115321_standard_comp_to_omission_D3_s...   
50  20230619_115321_standard_comp_to_omission_D3_s...   

                                            recording metadata_dir  \
0   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
1   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
2   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
3   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
4   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
5   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
6   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
7   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
8   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
9   20230617_115521_standard_comp_to_omission_D1_s...          DIO   
10  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
11  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
12  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
13  20230617_115521_standard_comp_to_omission_D1_s...          DIO   
14  20230617_115521_standar

In [69]:
trodes_state_df = trodes_state_df[STATE_COLS_TO_KEEP].drop_duplicates(subset=["session_dir", "video_name", "metadata_file"]).sort_values(["session_dir", "video_name", "metadata_file"]).reset_index(drop=True).copy()

In [70]:
trodes_state_df.head()

session_dir metadata_file  \
0  20230617_115521_standard_comp_to_omission_D1_s...  dio_ECU_Din1   
1  20230617_115521_standard_comp_to_omission_D1_s...  dio_ECU_Din2   
2  20230617_115521_standard_comp_to_omission_D1_s...  dio_ECU_Din3   
3  20230617_115521_standard_comp_to_omission_D1_s...  dio_ECU_Din1   
4  20230617_115521_standard_comp_to_omission_D1_s...  dio_ECU_Din2   

                                    event_timestamps  \
0  [[4509412, 4709411], [6909411, 7109411], [9209...   
1  [[3581011, 3583414], [3589014, 3626412], [3671...   
2  [[28509009, 34858606], [43086003, 43099810], [...   
3  [[4509412, 4709411], [6909411, 7109411], [9209...   
4  [[3581011, 3583414], [3589014, 3626412], [3671...   

                                          video_name  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                    video_timestamps  \
0  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
1  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
2  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
3  [3478531, 3479917, 3479917, 3481303, 3482689, ...   
4  [3478531, 3479917, 3479917, 3481303, 3482689, ...   

                                        event_frames  
0  [[1029, 1229], [3425, 3624], [5721, 5920], [75...  
1  [[102, 105], [110, 148], [193, 199], [200, 202...  
2  [[24984, 31321], [39533, 39547], [39554, 39559...  
3  [[1029, 1229], [3425, 3624], [5721, 5920], [75...  
4  [[102, 105], [111, 148], [193, 199], [200, 202...

In [71]:
trodes_state_df = trodes_state_df.groupby(same_columns).agg({**{col: 'first' for col in trodes_state_df.columns if col not in same_columns + different_columns}, **{col: lambda x: x.tolist() for col in different_columns}}).reset_index()

In [72]:
trodes_state_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230618_100636_standard_comp_to_omission_D2_s...   
4  20230618_100636_standard_comp_to_omission_D2_s...   

                                          video_name  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230618_100636_standard_comp_to_omission_D2_s...   
4  20230618_100636_standard_comp_to_omission_D2_s...   

                                    video_timestamps  \
0  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
1  [3478531, 3479917, 3479917, 3481303, 3482689, ...   
2  [44148259, 44149645, 44149645, 44151031, 44152...   
3  [835678, 837064, 838450, 838450, 839835, 84122...   
4  [835678, 837064, 838450, 838921, 839835, 84122...   

                                metadata_file  \
0  [dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]   
1  [dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]   
2  [dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]   
3  [dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]   
4  [dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]   

                                        event_frames  \
0  [[[1029, 1229], [3425, 3624], [5721, 5920], [7...   
1  [[[1029, 1229], [3425, 3624], [5721, 5920], [7...   
2  [[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, ...   
3  [[[1098, 1297], [3492, 3693], [5788, 5988], [7...   
4  [[[1098, 1297], [3492, 3693], [5788, 5988], [7...   

                                    event_timestamps  
0  [[[4509412, 4709411], [6909411, 7109411], [920...  
1  [[[4509412, 4709411], [6909411, 7109411], [920...  
2  [[[4509412, 4709411], [6909411, 7109411], [920...  
3  [[[1934930, 2134930], [4334936, 4534932], [663...  
4  [[[1934930, 2134930], [4334936, 4534932], [663...

In [73]:
trodes_state_df["tone_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[0])
trodes_state_df["box_1_port_entry_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[1])
trodes_state_df["box_2_port_entry_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[2])

trodes_state_df["tone_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[0])
trodes_state_df["box_1_port_entry_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[1])
trodes_state_df["box_2_port_entry_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[2])


In [74]:
trodes_state_df = trodes_state_df.drop(columns=["event_timestamps", "event_frames", "metadata_file"], errors="ignore")

In [75]:
trodes_state_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230618_100636_standard_comp_to_omission_D2_s...   
4  20230618_100636_standard_comp_to_omission_D2_s...   

                                          video_name  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230618_100636_standard_comp_to_omission_D2_s...   
4  20230618_100636_standard_comp_to_omission_D2_s...   

                                    video_timestamps  \
0  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
1  [3478531, 3479917, 3479917, 3481303, 3482689, ...   
2  [44148259, 44149645, 44149645, 44151031, 44152...   
3  [835678, 837064, 838450, 838450, 839835, 84122...   
4  [835678, 837064, 838450, 838921, 839835, 84122...   

                                     tone_timestamps  \
0  [[4509412, 4709411], [6909411, 7109411], [9209...   
1  [[4509412, 4709411], [6909411, 7109411], [9209...   
2  [[4509412, 4709411], [6909411, 7109411], [9209...   
3  [[1934930, 2134930], [4334936, 4534932], [6634...   
4  [[1934930, 2134930], [4334936, 4534932], [6634...   

                         box_1_port_entry_timestamps  \
0  [[3581011, 3583414], [3589014, 3626412], [3671...   
1  [[3581011, 3583414], [3589014, 3626412], [3671...   
2  [[3581011, 3583414], [3589014, 3626412], [3671...   
3  [[1624532, 1672532], [1674332, 1675730], [1676...   
4  [[1624532, 1672532], [1674332, 1675730], [1676...   

                         box_2_port_entry_timestamps  \
0  [[28509009, 34858606], [43086003, 43099810], [...   
1  [[28509009, 34858606], [43086003, 43099810], [...   
2  [[28509009, 34858606], [43086003, 43099810], [...   
3  [[33893117, 33905117], [33905717, 33910719], [...   
4  [[33893117, 33905117], [33905717, 33910719], [...   

                                         tone_frames  \
0  [[1029, 1229], [3425, 3624], [5721, 5920], [75...   
1  [[1029, 1229], [3425, 3624], [5721, 5920], [75...   
2  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...   
3  [[1098, 1297], [3492, 3693], [5788, 5988], [75...   
4  [[1098, 1297], [3492, 3693], [5788, 5988], [75...   

                             box_1_port_entry_frames  \
0  [[102, 105], [110, 148], [193, 199], [200, 202...   
1  [[102, 105], [111, 148], [193, 199], [200, 202...   
2  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...   
3  [[788, 835], [838, 839], [839, 871], [871, 900...   
4  [[788, 835], [838, 839], [839, 871], [871, 900...   

                             box_2_port_entry_frames  
0  [[24984, 31321], [39533, 39547], [39554, 39559...  
1  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...  
2  [[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...  
3  [[33026, 33037], [33038, 33042], [33044, 33049...  
4  [[32996, 33007], [33008, 33012], [33014, 33019...

In [76]:
trodes_raw_df = trodes_raw_df[RAW_COLS_TO_KEEP].reset_index(drop=True).copy()

In [77]:
trodes_raw_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230621_111240_standard_comp_to_omission_D5_s...   
3  20230620_114347_standard_comp_to_omission_D4_s...   
4  20230620_114347_standard_comp_to_omission_D4_s...   

                                           recording  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230621_111240_standard_comp_to_omission_D5_s...   
3  20230620_114347_standard_comp_to_omission_D4_s...   
4  20230620_114347_standard_comp_to_omission_D4_s...   

                                       original_file  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230621_111240_standard_comp_to_omission_D5_s...   
3  20230620_114347_standard_comp_to_omission_D4_s...   
4  20230620_114347_standard_comp_to_omission_D4_s...   

                                        session_path current_subject  \
0  /scratch/back_up/reward_competition_extention/...             1.1   
1  /scratch/back_up/reward_competition_extention/...             1.2   
2  /scratch/back_up/reward_competition_extention/...             1.4   
3  /scratch/back_up/reward_competition_extention/...             1.1   
4  /scratch/back_up/reward_competition_extention/...             1.2   

                                     first_item_data  first_timestamp  \
0  [3478533, 3478534, 3478535, 3478536, 3478537, ...          3478533   
1  [3478533, 3478534, 3478535, 3478536, 3478537, ...          3478533   
2  [3193044, 3193045, 3193046, 3193047, 3193048, ...          3193044   
3  [2502881, 2502882, 2502883, 2502884, 2502885, ...          2502881   
4  [2502881, 2502882, 2502883, 2502884, 2502885, ...          2502881   

  all_subjects  
0   [1.1, 1.2]  
1   [1.1, 1.2]  
2   [1.2, 1.4]  
3   [1.1, 1.2]  
4   [1.1, 1.2]

In [78]:
trodes_final_df = pd.merge(trodes_raw_df, trodes_state_df, on=["session_dir"], how="inner")

In [79]:
trodes_final_df.shape

(17, 16)

In [80]:
trodes_final_df = trodes_final_df.rename(columns={"first_item_data": "raw_timestamps"})
trodes_final_df = trodes_final_df.drop(columns=["metadata_file"], errors="ignore")
trodes_final_df = trodes_final_df.sort_values(["session_dir", "recording"]).reset_index(drop=True).copy()

## Making the timestamps 0 indexed

In [81]:
trodes_final_df[[col for col in trodes_final_df.columns if "timestamps" in col]].head()

raw_timestamps  \
0  [3478533, 3478534, 3478535, 3478536, 3478537, ...   
1  [3478533, 3478534, 3478535, 3478536, 3478537, ...   
2  [3478533, 3478534, 3478535, 3478536, 3478537, ...   
3  [3478533, 3478534, 3478535, 3478536, 3478537, ...   
4  [3478533, 3478534, 3478535, 3478536, 3478537, ...   

                                    video_timestamps  \
0  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
1  [3478531, 3479917, 3479917, 3481303, 3482689, ...   
2  [44148259, 44149645, 44149645, 44151031, 44152...   
3  [3478531, 3479917, 3480033, 3481303, 3482689, ...   
4  [3478531, 3479917, 3479917, 3481303, 3482689, ...   

                                     tone_timestamps  \
0  [[4509412, 4709411], [6909411, 7109411], [9209...   
1  [[4509412, 4709411], [6909411, 7109411], [9209...   
2  [[4509412, 4709411], [6909411, 7109411], [9209...   
3  [[4509412, 4709411], [6909411, 7109411], [9209...   
4  [[4509412, 4709411], [6909411, 7109411], [9209...   

                         box_1_port_entry_timestamps  \
0  [[3581011, 3583414], [3589014, 3626412], [3671...   
1  [[3581011, 3583414], [3589014, 3626412], [3671...   
2  [[3581011, 3583414], [3589014, 3626412], [3671...   
3  [[3581011, 3583414], [3589014, 3626412], [3671...   
4  [[3581011, 3583414], [3589014, 3626412], [3671...   

                         box_2_port_entry_timestamps  
0  [[28509009, 34858606], [43086003, 43099810], [...  
1  [[28509009, 34858606], [43086003, 43099810], [...  
2  [[28509009, 34858606], [43086003, 43099810], [...  
3  [[28509009, 34858606], [43086003, 43099810], [...  
4  [[28509009, 34858606], [43086003, 43099810], [...

In [82]:
trodes_final_df["last_timestamp"] = trodes_final_df["raw_timestamps"].apply(lambda x: x[-1])

- Dropping raw timestamps because of memory issues

In [83]:
trodes_final_df = trodes_final_df.drop(columns=["raw_timestamps", "original_file"], errors="ignore")

In [84]:
copy_trodes_final_df = trodes_final_df.copy

In [85]:
for col in [col for col in trodes_final_df.columns if "timestamps" in col]:
    trodes_final_df[col] = trodes_final_df.apply(lambda x: x[col].astype(np.int32) - np.int32(x["first_timestamp"]), axis=1)

for col in [col for col in trodes_final_df.columns if "frames" in col]:
    trodes_final_df[col] = trodes_final_df[col].apply(lambda x: x.astype(np.int32))

In [86]:
sorted_columns = sorted(trodes_final_df.columns
, key=lambda x: x.split("_")[-1])
trodes_final_df = trodes_final_df[sorted_columns].copy()

## Saving to a file

In [87]:
trodes_final_df.to_pickle(os.path.join(OUTPUT_DIR, "{}_00_trodes_metadata.pkl".format(OUTPUT_PREFIX)))

In [88]:
trodes_final_df.head()

session_dir  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                         tone_frames  \
0  [[1029, 1229], [3425, 3624], [5721, 5920], [75...   
1  [[1029, 1229], [3425, 3624], [5721, 5920], [75...   
2  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...   
3  [[1029, 1229], [3425, 3624], [5721, 5920], [75...   
4  [[1029, 1229], [3425, 3624], [5721, 5920], [75...   

                             box_1_port_entry_frames  \
0  [[102, 105], [110, 148], [193, 199], [200, 202...   
1  [[102, 105], [111, 148], [193, 199], [200, 202...   
2  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...   
3  [[102, 105], [110, 148], [193, 199], [200, 202...   
4  [[102, 105], [111, 148], [193, 199], [200, 202...   

                             box_2_port_entry_frames  \
0  [[24984, 31321], [39533, 39547], [39554, 39559...   
1  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...   
2  [[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...   
3  [[24984, 31321], [39533, 39547], [39554, 39559...   
4  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...   

                                          video_name  \
0  20230617_115521_standard_comp_to_omission_D1_s...   
1  20230617_115521_standard_comp_to_omission_D1_s...   
2  20230617_115521_standard_comp_to_omission_D1_s...   
3  20230617_115521_standard_comp_to_omission_D1_s...   
4  20230617_115521_standard_comp_to_omission_D1_s...   

                                        session_path  \
0  /scratch/back_up/reward_competition_extention/...   
1  /scratch/back_up/reward_competition_extention/...   
2  /scratch/back_up/reward_competition_extention/...   
3  /scratch/back_up/reward_competition_extention/...   
4  /scratch/back_up/reward_competition_extention/...   

                                           recording current_subject  \
0  20230617_115521_standard_comp_to_omission_D1_s...             1.1   
1  20230617_115521_standard_comp_to_omission_D1_s...             1.1   
2  20230617_115521_standard_comp_to_omission_D1_s...             1.1   
3  20230617_115521_standard_comp_to_omission_D1_s...             1.2   
4  20230617_115521_standard_comp_to_omission_D1_s...             1.2   

  all_subjects  first_timestamp  last_timestamp  \
0   [1.1, 1.2]          3478533        83177118   
1   [1.1, 1.2]          3478533        83177118   
2   [1.1, 1.2]          3478533        83177118   
3   [1.1, 1.2]          3478533        83177118   
4   [1.1, 1.2]          3478533        83177118   

                                    video_timestamps  \
0  [-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...   
1  [-2, 1384, 1384, 2770, 4156, 5542, 5542, 6927,...   
2  [40669726, 40671112, 40671112, 40672498, 40673...   
3  [-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...   
4  [-2, 1384, 1384, 2770, 4156, 5542, 5542, 6927,...   

                                     tone_timestamps  \
0  [[1030879, 1230878], [3430878, 3630878], [5730...   
1  [[1030879, 1230878], [3430878, 3630878], [5730...   
2  [[1030879, 1230878], [3430878, 3630878], [5730...   
3  [[1030879, 1230878], [3430878, 3630878], [5730...   
4  [[1030879, 1230878], [3430878, 3630878], [5730...   

                         box_1_port_entry_timestamps  \
0  [[102478, 104881], [110481, 147879], [193083, ...   
1  [[102478, 104881], [110481, 147879], [193083, ...   
2  [[102478, 104881], [110481, 147879], [193083, ...   
3  [[102478, 104881], [110481, 147879], [193083, ...   
4  [[102478, 104881], [110481, 147879], [193083, ...   

                         box_2_port_entry_timestamps  
0  [[25030476, 31380073], [39607470, 39621277], [...  
1  [[25030476, 31380073], [39607470, 39621277], [...  
2  [[25030476, 31380073], [39607470, 39621277], [...  
3  [[25030476, 31380073], [39607470, 39621277], 

In [89]:
trodes_final_df["session_dir"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2'],
      dtype=object)

In [90]:
trodes_final_df["video_name"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.2.videoTimeStamps.cameraHWSync',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3.videoTimeStamps.cameraHWSync',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1.videoTimeStamps.cameraHWSync',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2.videoTimeStamps.cameraHWSync',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.4.videoTimeStamps.cameraHWSync',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1.videoTimeStamps.cameraHWSync',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.2.videoTimeStamps.cameraHWSync',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.2.v